In [9]:
import re
import pandas as pd
import pymupdf4llm
from pathlib import Path
from natsort import natsorted
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
#set API Key from OpenAI
openai_api_key= "Add Your OpenAI API KEY Here."

In [ ]:
def load_pdf_paths(pdf_directory, base_persist_directory):
    """Load PDF file paths in natural order and assign persist directories in groups of three with local IDs (1,2,3) per batch."""    
    files = [file for file in os.listdir(pdf_directory) if file.endswith(".pdf")]
    sorted_files = natsorted(files)  
    
    all_files = [str(Path(pdf_directory, file).as_posix()) for file in sorted_files]
    
    num_of_papers = len(all_files)  

    document_ids = list(range(1, num_of_papers + 1))

    persist_directories = [
        str(Path(base_persist_directory, f"db_{doc_id}").as_posix())
        for doc_id in document_ids
    ]
    
    df_pdf = pd.DataFrame({
        'Document_ID': document_ids, 
        'PDF_path': all_files, 
        'Vectordb_path': persist_directories
    })
    
    return df_pdf
    
def load_the_document(pdf_path):
    """Loads the document and extracts text content"""
    # Load document
    docs = pymupdf4llm.to_markdown(pdf_path, page_chunks=True)

    # Extract metadata
    paper_title = docs[0]['metadata'].get('title', 'Unknown Title')
    num_pages = docs[0]['metadata'].get('page_count', 0)
    author = docs[0]['metadata'].get('author', 'Unknown Author')

    pages_content = []

    for i, page in enumerate(docs, start=1):
        page_text = remove_reference(page['text']).strip()
        pages_content.append({
            "page_number": i,
            "text": page_text
        })

    return pages_content, paper_title, author, num_pages


def remove_reference(pdf_text):
    """Removes references and acknowledgment sections, along with inline citations, from a given PDF text."""
    # Remove references and acknowledgment sections entirely
    pdf_text = re.split(r'(?i)\bReferences\b|\bAcknowledgment[s]?\b', pdf_text)[0].strip()

    # Remove inline citations like [1], [12-15], (Smith et al., 2021), etc.
    citation_patterns = [
        r'\[[^\]]*\d{1,4}\]',          # numeric citations like [1], [12], [ABC12]
        r'\([^\)]*et al\.,?\s*\d{4}[^\)]*\)',  # (Author, 2020)
        r'\(\s*\d{4}[a-z]?\s*\)',  # (2021a)
    ]

    for pattern in citation_patterns:
        pdf_text = re.sub(pattern, '', pdf_text)

    # Clean up extra spaces and blank lines
    pdf_text = '\n'.join(line.strip() for line in pdf_text.splitlines() if line.strip())

    return pdf_text


def embedding_document(pages_content, ID, persist_directory):
    """Embeds documents and stores them persistently in a vector database (Chroma), recording page number."""

    documents = []
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=3000,   # small for better retrieval precision
        chunk_overlap=400,
        length_function=len,
        separators=["\n\n", "\n", "."]
    )

    document_id = ID

    for page in pages_content:
        page_number = page["page_number"]
        page_text = page["text"]

        # Split this page into smaller chunks
        page_chunks = text_splitter.create_documents([page_text])

        for chunk in page_chunks:
            documents.append(
                Document(
                    page_content=chunk.page_content,
                    metadata={
                        "page_number": page_number,
                        "document_id": document_id 
                    }
                )
            )

    num_chunks = len(documents)

    # Initialize embeddings
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

    vectordb = Chroma.from_documents(
        documents=documents,
        embedding=embeddings, 
        persist_directory=persist_directory
    )

    vectordb.persist()

    return num_chunks

def build_vectorDB(df_path):
    """Processes each PDF, extracts metadata, builds vector DB, and updates DataFrame."""
    success_status = []  # Track success for each PDF
    paper_titles = []  # Store paper titles
    author_list = []  # Store author information
    num_pages_list = []  # Store number of pages
    chunk_nums = []  # Store chunk numbers separately

    for _, row in df_path.iterrows():
        pdf_path = row["PDF_path"]
        ID = row["Document_ID"]
        persist_directory = row["Vectordb_path"]
        
        try:
            # Extract content, title, and page count
            docs, paper_title, author, num_pages = load_the_document(pdf_path)

            # Build the vector database
            num_chunks = embedding_document(docs, ID, persist_directory)  # Returns an integer

            success_status.append(True)

            # Store extracted metadata
            paper_titles.append(paper_title)
            author_list.append(author)
            num_pages_list.append(num_pages)
            chunk_nums.append(num_chunks) 

            print(f"Successfully processed {pdf_path}")
        
        except Exception as e:
            print(f"Error processing {pdf_path}: {e}")
            success_status.append(False)  # Mark as failed

            # Store placeholders for failed cases
            paper_titles.append(None)
            author_list.append(None)
            num_pages_list.append(None)
            chunk_nums.append(None)  
            vectordb = None  # Ensure vectordb is always defined

    # Add extracted metadata to the DataFrame
    df_path["Paper_title"] = paper_titles
    df_path["Author"] = author_list
    df_path["Num_pages"] = num_pages_list
    df_path["Num_chunks"] = chunk_nums  
    df_path["Is_added_to_vectorDB"] = success_status

    return df_path

def df_to_csv(df, file_name):
    """Write a DataFrame to a CSV file"""
    df.to_csv(file_name, index=False, escapechar='\\')

In [ ]:
# Directory containing source PDF files to be processed
pdf_directory = "Add your PDF directory path here"
# Directory where vector databases are persisted
base_persist_directory = "Add your vector DB directory path here"

# Load PDF file paths
df_path = load_pdf_paths(pdf_directory, base_persist_directory)
# Build the vector database
df = build_vectorDB(df_path)
df

In [ ]:
# Output file path for saving the vectordb and metadata
file_path = "Add your output CSV path here"   # This file will be used in Steps 2 and 3
df_to_csv(df, file_path)